##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tunagem de hiperparâmetros com o painel de controle HParams

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tensorboard/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tensorboard/hyperparameter_tuning_with_hparams.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Ao criar modelos de aprendizado de máquina, você precisa escolher vários [hiperparâmetros](https://en.wikipedia.org/wiki/Hyperparameter_(machine_learning)), como a taxa de dropout em uma camada ou a taxa de aprendizado. Essas decisões afetam as métricas dos modelos, como exatidão. Portanto, um passo importante no fluxo de trabalho do aprendizado de máquina é identificar os melhores hiperparâmetros para seu problema, o que geralmente envolve experimentação. Esse processo é conhecido como "otimização de hiperparâmetros" ou "tunagem de hiperparâmetros".

O painel de controle HParams no TensorBoard oferece várias ferramentas para ajudar com esse processo de identificar o melhor experimento ou os conjuntos mais promissores de hiperparâmetros.

Este tutorial focará nas seguintes etapas:

1. Testar a configuração e o resumo do HParams
2. Adaptar as execuções do TensorFlow para registrar hiperparâmetros e métricas
3. Iniciar as execuções e registrar todas elas em um diretório pai
4. Visualizar os resultados no painel de controle HParams do TensorBoard

Observação: as APIs de resumo do HParams e a interface de usuário do painel de controle estão em fase de prévia e mudarão ao longo do tempo.

Comece instalando o TF 2.0 e carregando a extensão de notebook do TensorBoard:

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

Importe o TensorFlow e o plugin HParams do TensorBoard:

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

Baixe e dimensione o dataset [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist):

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

4423680/4422102 [==============================] - 0s 0us/step


## 1. Teste a configuração e o resumo do experimento HParams

Teste três hiperparâmetros no modelo:

1. Número de unidades na primeira camada densa
2. Taxa de dropout na camada de dropout
3. Otimizador

Liste os valores que serão testados e registre uma configuração de experimento para o TensorBoard. Esse passo é opcional: você pode oferecer informações de domínio para permitir uma filtragem mais precisa de hiperparâmetros na interface do usuário e também é possível especificar quais métricas devem ser exibidas.

In [ ]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16, 32]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

Se você decidir pular essa etapa, poderá utilizar um literal de string onde usaria um valor `HParam`: por exemplo, `hparams['dropout']` em vez de `hparams[HP_DROPOUT]`.

## 2. Adapte as execuções do TensorFlow para registrar hiperparâmetros e métricas

O modelo será bastante simples: duas camadas densas com uma camada de dropout entre elas. O código de treinamento parecerá familiar, embora os hiperparâmetros não sejam mais embutidos no código. Em vez disso, os hiperparâmetros são fornecidos em um dicionário `hparams` e usados durante a função de treinamento:

In [ ]:
def train_test_model(hparams):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
    tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
  ])
  model.compile(
      optimizer=hparams[HP_OPTIMIZER],
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'],
  )

  model.fit(x_train, y_train, epochs=1) # Run with 1 epoch to speed things up for demo purposes
  _, accuracy = model.evaluate(x_test, y_test)
  return accuracy

Para cada execução, registre um resumo de hparams com os hiperparâmetros e a exatidão final:

In [ ]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

Ao treinar modelos Keras, você pode usar callbacks em vez de escrever isto diretamente:

```python
model.fit(
    ...,
    callbacks=[
        tf.keras.callbacks.TensorBoard(logdir),  # log metrics
        hp.KerasCallback(logdir, hparams),  # log hparams
    ],
)
```

## 3. Inicie as execuções e registre todas elas em um diretório pai

Agora você pode fazer vários experimentos, treinando cada um com um conjunto diferente de hiperparâmetros.

Para simplificar, use uma busca em grade: teste todas as combinações de parâmetros discretos e só os limites inferiores e superiores do parâmetro real. Para cenários mais complexos, pode ser mais eficaz escolher cada valor de hiperparâmetro aleatoriamente (isso é chamado de busca aleatória). Há métodos mais avançados que podem ser usados.

Execute alguns experimentos, o que levará alguns minutos:

In [ ]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
  for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_NUM_UNITS: num_units,
          HP_DROPOUT: dropout_rate,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


--- Starting trial: run-0
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 35us/sample - loss: 0.4806 - accuracy: 0.8321
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.1, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.6519 - accuracy: 0.7770
--- Starting trial: run-2
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.5309 - accuracy: 0.8154
--- Starting trial: run-3
{'num_units': 16, 'dropout': 0.2, 'optimizer': 'sgd'}
10000/10000 [==============================] - 0s 36us/sample - loss: 0.7007 - accuracy: 0.7683
--- Starting trial: run-4
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'adam'}
10000/10000 [==============================] - 0s 38us/sample - loss: 0.4645 - accuracy: 0.8395
--- Starting trial: run-5
{'num_units': 32, 'dropout': 0.1, 'optimizer': 'sgd'}
10000/10000 [========================

## 4. Visualize os resultados no plugin HParams do TensorBoard

O painel de controle HParams já pode ser aberto. Inicie o TensorBoard e clique em "HParams" na parte superior.

In [ ]:
%tensorboard --logdir logs/hparam_tuning

<!-- <img class="tfo-display-only-on-site" src="images/hparams_table.png?raw=1"/> -->

A janela à esquerda do painel de controle fornece capacidades de filtragem que ficam ativas em todas as visualizações no painel de controle HParams:

- Filtre quais hiperparâmetros/métricas são exibidos no painel de controle
- Filtre quais valores de hiperparâmetros/métricas são exibidos no painel de controle
- Filtre pelo status da execução (em andamento, sucesso...)
- Classifique por hiperparâmetro/métrica na visualização de tabela
- Número de grupos de sessão exibidos (útil para o desempenho quando há vários experimentos)


O painel de controle HParams tem três visualizações diferentes, com várias informações úteis:

- A **Table View** (Visualização de tabela) lista as execuções, os hiperparâmetros e as métricas.
- A **Parallel Coordinates View** (Visualização de coordenadas paralelas) mostra cada execução como uma linha que percorre um eixo para cada hiperparâmetro e métrica. Clique e arraste o mouse em qualquer eixo para marcar uma região, destacando somente as execuções que passarem por ela. Isso pode ser útil para identificar quais grupos de hiperparâmetros são mais importantes. Os próprios eixos podem ser reordenados ao soltá-los.
- A **Scatter Plot View** (Visualização de plotagem de dispersão) mostra plotagens que comparam cada hiperparâmetro/métrica a cada métrica. Isso pode ajudar a identificar correlações. Clique e arraste para selecionar uma região em uma plotagem específica e destacar essas sessões em todas as outras plotagens.

É possível clicar em uma linha da tabela, uma linha das coordenadas paralelas e um mercado da plotagem de dispersão para ver uma plotagem das métricas como uma função dos passos de treinamento para essa sessão (embora neste tutorial somente um passo seja usado para cada execução).

Para explorar ainda mais as capacidades do painel de controle HParams, baixe um conjunto de logs pré-gerados com mais experimentos:

In [ ]:
%%bash
wget -q 'https://storage.googleapis.com/download.tensorflow.org/tensorboard/hparams_demo_logs.zip'
unzip -q hparams_demo_logs.zip -d logs/hparam_demo

Veja esses logs no TensorBoard:

In [ ]:
%tensorboard --logdir logs/hparam_demo

<!-- <img class="tfo-display-only-on-site" src="images/hparams_parallel_coordinates.png?raw=1"/> -->

Você pode testar as diferentes visualizações no painel de controle HParams.

Por exemplo, ao acessar a visualização de coordenadas paralelas e clicar e arrastar o eixo de exatidão, você pode selecionar as execuções com maior exatidão. Conforme essas execuções passarem por "adam" no eixo de otimizador, você pode concluir que o "adam" teve um melhor desempenho do que "sgd" nesses experimentos.